In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import json
import re
from collections import Counter

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:

with open('../input/train.json', 'r') as f:
    train = json.load(f)
with open('../input/test.json', 'r') as f:
    test = json.load(f)

In [ ]:
SPEC_CHARS = re.compile(r'[^\w\s_]')

chars = [re.findall(SPEC_CHARS, x)\
 for ilist in [r['ingredients'] for r in train+test] for x in ilist if re.search(SPEC_CHARS, x)]

Counter([x for charlist in chars for x in charlist])

In [ ]:
SPEC_REMOVE = re.compile(r'(\'|\’|\(.*oz.*\)|(\()|(\))|\™|\®|\.|\€|\-)')
SPEC_AND = re.compile(r'\&')
SPEC_ELSE = re.compile(r'[^\w\s\%_]')

def clean_ingr(ingr):
    ingr = re.sub(SPEC_REMOVE, '', ingr)
    ingr = re.sub(SPEC_AND, 'and', ingr)
    ingr = re.sub(SPEC_ELSE, ' ', ingr)
    return ' '.join(ingr.split())

def get_ingrs(given):
    ingrs = [[clean_ingr(i).lower() for i in recipe['ingredients']] for recipe in given]
    return ingrs

def get_labels(given):
    return [r['cuisine'] for r in given]

In [ ]:
train_ingrs = get_ingrs(train)
train_labels = get_labels(train)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [ ]:
def itself(x):
    return x

def combine_words(ilist):
    return ' '.join(ilist)

In [ ]:
ingr_word = Pipeline([
    ('union', FeatureUnion([
        ("ingrs", TfidfVectorizer(strip_accents='unicode',
                                  tokenizer=itself,
                                  preprocessor=itself)),
        ("words", TfidfVectorizer(strip_accents='unicode',
                                  preprocessor=combine_words,
                                  stop_words=None,
                                  ngram_range=(1, 4)))
    ])),
    ("linear svc", LinearSVC(loss='hinge', C=10**0.1))
])

In [ ]:
%%time
scores = cross_val_score(ingr_word, train_ingrs, train_labels, cv=5, n_jobs=-1)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
%%time
dvec_all = FeatureUnion([
        ("ingrs", TfidfVectorizer(strip_accents='unicode',
                                  tokenizer=itself,
                                  preprocessor=itself)),
        ("words", TfidfVectorizer(strip_accents='unicode',
                                  preprocessor=combine_words,
                                  ngram_range=(1, 4),
                                  stop_words=None)),
        ]).fit(get_ingrs(train+test))

In [ ]:
test_bag = dvec_all.transform(get_ingrs(test))

In [ ]:
svc_linear =  LinearSVC(loss='hinge', C=10**0.1)

In [ ]:
%%time
svc_linear = svc_linear.fit(dvec_all.transform(train_ingrs), train_labels)

In [ ]:
test_ids = [r['id'] for r in test]
test_preds = svc_linear.predict(dvec_all.transform(get_ingrs(test)))

In [ ]:
df_test = pd.DataFrame({'id': test_ids, 'cuisine': test_preds}, columns=['id', 'cuisine'])

In [ ]:
df_test.to_csv('submission.csv', index=False)